#### sample request to make sure we can hit the ES server:

In [2]:
# make sure ES is up and running
import requests
res = requests.get('http://localhost:9200')
print(res.content)

{
  "name" : "BwZDr-h",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "6pfWlUxGRsqoSM5y4--HJA",
  "version" : {
    "number" : "5.2.2",
    "build_hash" : "f9d9b74",
    "build_date" : "2017-02-24T17:26:45.835Z",
    "build_snapshot" : false,
    "lucene_version" : "6.4.1"
  },
  "tagline" : "You Know, for Search"
}



#### Then we connect to our ES server using Python and the elasticsearch-py library:

In [4]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [5]:
#index some test data
es.index(index='test-index', doc_type='test', id=1, body={'test': 'test'})

{u'_id': u'1',
 u'_index': u'test-index',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'test',
 u'_version': 1,
 u'created': True,
 u'result': u'created'}

In [6]:
#delete test data and try with something more interesting
es.delete(index='test-index', doc_type='test', id=1)

{u'_id': u'1',
 u'_index': u'test-index',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'test',
 u'_version': 2,
 u'found': True,
 u'result': u'deleted'}

In [7]:
es.index(index='sw', doc_type='people', id=1, body={
	"name": "Luke Skywalker",
	"height": "172",
	"mass": "77",
	"hair_color": "blond",
	"birth_year": "19BBY",
	"gender": "male",
})

{u'_id': u'1',
 u'_index': u'sw',
 u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
 u'_type': u'people',
 u'_version': 1,
 u'created': True,
 u'result': u'created'}

#### Now, using The Force, we connect to the Star Wars API and index some fictional people.

In [8]:
#let's iterate over swapi people documents and index them
import json
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
 
print(i)

18


#### Anyway, to see if all worked with this few results, we try to get the document with id=5.

In [9]:
es.get(index='sw', doc_type='people', id=5)

{u'_id': u'5',
 u'_index': u'sw',
 u'_source': {u'birth_year': u'19BBY',
  u'created': u'2014-12-10T15:20:09.791000Z',
  u'edited': u'2014-12-20T21:17:50.315000Z',
  u'eye_color': u'brown',
  u'films': [u'http://swapi.co/api/films/6/',
   u'http://swapi.co/api/films/3/',
   u'http://swapi.co/api/films/2/',
   u'http://swapi.co/api/films/1/',
   u'http://swapi.co/api/films/7/'],
  u'gender': u'female',
  u'hair_color': u'brown',
  u'height': u'150',
  u'homeworld': u'http://swapi.co/api/planets/2/',
  u'mass': u'49',
  u'name': u'Leia Organa',
  u'skin_color': u'light',
  u'species': [u'http://swapi.co/api/species/1/'],
  u'starships': [],
  u'url': u'http://swapi.co/api/people/5/',
  u'vehicles': [u'http://swapi.co/api/vehicles/30/']},
 u'_type': u'people',
 u'_version': 1,
 u'found': True}

In [ ]:
r = requests.get('http://localhost:9200')
i = 18
while r.status_code == 200:
   r = requests.get('http://swapi.co/api/people/'+ str(i))
   es.index(index='sw', doc_type='people', id=i,     body=json.loads(r.content))
   i=i+1

print i

In [11]:
es.search(index="sw", body={"query": {"match": {'name':'Darth Vader'}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'4',
    u'_index': u'sw',
    u'_score': 4.082477,
    u'_source': {u'birth_year': u'41.9BBY',
     u'created': u'2014-12-10T15:18:20.704000Z',
     u'edited': u'2014-12-20T21:17:50.313000Z',
     u'eye_color': u'yellow',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/'],
     u'gender': u'male',
     u'hair_color': u'none',
     u'height': u'202',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'136',
     u'name': u'Darth Vader',
     u'skin_color': u'white',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/13/'],
     u'url': u'http://swapi.co/api/people/4/',
     u'vehicles': []},
    u'_type': u'people'},
   {u'_id': u'44',
    u'_index': u'sw',
    u'_score': 2.0412385,
    u'_source': {u'birth_year

#### There are many ways to do that and another great number of queries. Take a look here to learn more. I picked this one to get all documents with prefix “lu” in their name field:

In [13]:
es.search(index="sw", body={"query": {"prefix" : { "name" : "lu" }}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'1',
    u'_index': u'sw',
    u'_score': 1.0,
    u'_source': {u'birth_year': u'19BBY',
     u'created': u'2014-12-09T13:50:51.644000Z',
     u'edited': u'2014-12-20T21:17:56.891000Z',
     u'eye_color': u'blue',
     u'films': [u'http://swapi.co/api/films/6/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/2/',
      u'http://swapi.co/api/films/1/',
      u'http://swapi.co/api/films/7/'],
     u'gender': u'male',
     u'hair_color': u'blond',
     u'height': u'172',
     u'homeworld': u'http://swapi.co/api/planets/1/',
     u'mass': u'77',
     u'name': u'Luke Skywalker',
     u'skin_color': u'fair',
     u'species': [u'http://swapi.co/api/species/1/'],
     u'starships': [u'http://swapi.co/api/starships/12/',
      u'http://swapi.co/api/starships/22/'],
     u'url': u'http://swapi.co/api/people/1/',
     u'vehicles': [u'http://swapi.co/api/vehicles/14/',
      u'http:

#### If, for example, we want to get all elements similar in some way, for a related or correction search we can use something like this:

In [16]:
es.search(index="sw", body={"query": 
                            {"multi_match": 
                             {"fields": ["name"], "query": "jaba", "fuzziness": "AUTO"}}})

{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [{u'_id': u'16',
    u'_index': u'sw',
    u'_score': 1.4263155,
    u'_source': {u'birth_year': u'600BBY',
     u'created': u'2014-12-10T17:11:31.638000Z',
     u'edited': u'2014-12-20T21:17:50.338000Z',
     u'eye_color': u'orange',
     u'films': [u'http://swapi.co/api/films/4/',
      u'http://swapi.co/api/films/3/',
      u'http://swapi.co/api/films/1/'],
     u'gender': u'hermaphrodite',
     u'hair_color': u'n/a',
     u'height': u'175',
     u'homeworld': u'http://swapi.co/api/planets/24/',
     u'mass': u'1,358',
     u'name': u'Jabba Desilijic Tiure',
     u'skin_color': u'green-tan, brown',
     u'species': [u'http://swapi.co/api/species/5/'],
     u'starships': [],
     u'url': u'http://swapi.co/api/people/16/',
     u'vehicles': []},
    u'_type': u'people'}],
  u'max_score': 1.4263155,
  u'total': 1},
 u'timed_out': False,
 u'took': 71}